In [11]:
 #!/usr/bin/env python3

## Import Python packages for plotting GEE data in Leaflet

In [12]:
from oauth2client import crypt

In [13]:
# import packages
import pandas as pd
import json
import sys
import ee 

ee.Initialize()

import folium
print(folium.__version__)
sys.path.append('..')
print (folium.__file__)
print (folium.__version__)

0.6.0
/opt/conda/lib/python3.6/site-packages/folium/__init__.py
0.6.0


### Create definition files for working with Google Earth Engine

In [14]:
def folium_gee_map(image,vis_params=None,folium_kwargs={}):
    """
    Function to view Google Earth Engine tile layer as a Folium map.
    
    Parameters
    ----------
    image : Google Earth Engine Image.
    vis_params : Dict with visualization parameters.
    folium_kwargs : Keyword args for Folium Map.
    """
    
    # Get the MapID and Token after applying parameters
    image_info = image.getMapId(vis_params)
    mapid = image_info['mapid']
    token = image_info['token']
    folium_kwargs['attr'] = ('Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a> ')
    folium_kwargs['tiles'] = "https://earthengine.googleapis.com/map/%s/{z}/{x}/{y}?token=%s"%(mapid,token)
    
    return folium.Map(**folium_kwargs)

def folium_gee_layer(folium_map,image,vis_params=None,folium_kwargs={}):
    """
    Function to add Google Earch Engine tile layer as a Folium layer.
    
    Parameters
    ----------
    folium_map : Folium map to add tile to.
    image : Google Earth Engine Image.
    vis_params : Dict with visualization parameters.
    folium_kwargs : Keyword args for Folium Map.
    """
    
    # Get the MapID and Token after applying parameters
    image_info = image.getMapId(vis_params)
    mapid = image_info['mapid']
    token = image_info['token']
    folium_kwargs['attr'] = ('Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a> ')
    folium_kwargs['tiles'] = "https://earthengine.googleapis.com/map/%s/{z}/{x}/{y}?token=%s"%(mapid,token) 
    layer = folium.TileLayer(**folium_kwargs)
    layer.add_to(folium_map)

# Plot Leaflet Map over the Santa Rita Experimental Range site 

We are also adding the Landsat 8 TOA data with a visible band combination.

In [18]:
# Get an area to look at
lat = -18.800
lon = 14.865
zoom_start=11

# Open Street Map Base
m = folium.Map(location=[lat, lon], tiles="OpenStreetMap", zoom_start=zoom_start)

# Create a reference to the LANDSAT 8 image collection
l8 = ee.ImageCollection('LANDSAT/LC8_L1T_TOA')
# Filter the LANDSAT collection down to a eight month period
filtered = l8.filterDate('2016-12-01', '2017-03-31');
# Use the mosaic reducer, to select the most recent pixel in areas of overlap
l8_image = filtered.median()
l8_vis_params = {'min': 0, 'max':0.3, 'bands':'B4,B3,B2'}
folium_gee_layer(m,l8_image,l8_vis_params,folium_kwargs={'overlay':True,'name':'LANDSAT'})

# Create a reference to the 2016 sUAS orthophotography imager collection

m.add_child(folium.LayerControl())
m